In [2]:
import re
import codecs
import jieba
import pickle
import pandas as pd
from elasticsearch import Elasticsearch

In [3]:
es = Elasticsearch([{'host':'localhost','port':9200}])

In [6]:
# all_question = pd.read_csv('./data/description.csv')
Herceptin_question = pd.read_excel('./data/Herceptin.xlsx')
# all_question = pd.read_excel('./data/Herceptin_funding_filtered.xlsx')
# all_question = pd.read_excel('./data/Herceptin_patient_filtered.xlsx')
Kadcyla_question = pd.read_excel('./data/Kadcyla.xlsx')
Perjeta_1_question = pd.read_excel('./data/Perjeta_1.xlsx')
Perjeta_2_question = pd.read_excel('./data/Perjeta_2.xlsx')
Tecentriq_question = pd.read_excel('./data/Tecentriq.xlsx')
Xeloda_question = pd.read_excel('./data/Xeloda.xlsx')

all_question = pd.concat([Herceptin_question, Kadcyla_question, Perjeta_1_question, Perjeta_2_question, Tecentriq_question, Xeloda_question])
print('all_question: ', len(all_question))
all_question.head(5)

all_question:  1407


,Case Number,Description
0,1420656.0,"赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配..."
1,1423956.0,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用（无产品批号信息）
2,1434184.0,[Wechat] 请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献（确认无AE）
3,1436011.0,拿到的赫赛汀呈现块状是否正常
4,1438071.0,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用


In [7]:
all_question.dropna(axis=0, inplace=True, subset=['Description'])

In [8]:
all_question.isna().sum()

Case Number    0
Description    0
dtype: int64

In [11]:
jieba.load_userdict('./dict data/chinese_word_dict.txt')
chinese_word_dict = pickle.load(open('./dict data/chinese_word_dict.pickle', 'rb'))
stopwords = [line.strip() for line in codecs.open('./stopwords-zh.txt', 'r', encoding='utf8').readlines()]

In [12]:
def keyword_extract(text, word_dict):
#     text = list(jieba.cut(text, cut_all=False))
    text = list(jieba.cut_for_search(text))
    union_text = list(set(text).intersection(set(word_dict)))
    return " ".join(union_text)

In [13]:
all_question['KeyWords'] = all_question.Description.apply(lambda x: keyword_extract(x, chinese_word_dict))

In [14]:
# 去除所有“[]”内的文本
def remove_special_string(text):
    text = re.sub(r'\[[^]]*\]', '', text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'（[^）]*\）', '', text)
    text = re.sub(r'\<[^>]*\>', '', text)
    text = re.sub(r'[0-9]*', '', text)
    return text
def remove_stopwords(text, stopwords):
    text_cutted = jieba.cut(text, cut_all=True)
    text_removed = []
    for word in text_cutted:
        if word in stopwords or word.strip()=="":
            continue
        text_removed.append(word)
    return "  ".join(text_removed)

In [15]:
all_question['Processed'] = all_question.Description.apply(lambda x: remove_special_string(x))
all_question['Processed'] = all_question.Processed.apply(lambda x: remove_stopwords(x, stopwords))
all_question.head()

,Case Number,Description,KeyWords,Processed
0,1420656.0,"赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配...",赫赛汀 使用,配 之后 结冰 继续 使用
1,1423956.0,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用（无产品批号信息）,患者 无 赫赛汀 使用,粉末 形状 保存 稀释 稀释液 出现 结冰 冰情 情况 是否 使用
2,1434184.0,[Wechat] 请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献（确认无AE）,无 血栓 文献 血栓栓塞 查 栓塞 赫赛汀,帮忙 查 血栓 血栓栓塞 栓塞 方面 数据 和文 文献
3,1436011.0,拿到的赫赛汀呈现块状是否正常,赫赛汀 正常,拿到 呈现 块状 是否 正常
4,1438071.0,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用,使用 现 溶液 配置 赫赛汀 下,常规 配置 保存 复 溶 溶液 保存 存在 度 情况 现出 出现 浑...


In [16]:
doc_list = []
for idx,row in all_question.iterrows():
    doc = {"text": row['Description'], "processed_text": row['Processed']}
    doc_list.append(doc)
print(len(doc_list))
print(doc_list[0])

1093
{'text': '赫赛汀 （批号1：\xa0N3825B03B3049, 批号2： N3826B02B3050） 配了之后结冰了， 能否继续使用。', 'processed_text': '配  之后  结冰  继续  使用'}


In [17]:
for idx in range(len(doc_list)):
    es.index(index='index', doc_type='type', id=idx, body=doc_list[idx])

In [35]:

query = "关于赫赛汀心脏毒性：1、临床 2、临床前"

results = es.search(index='index', body={'query': {'match':{"text": query}}}, size=10)
hits = results['hits']['hits']
results
# for idx, res in enumerate(hits):
#     print(str(idx+1) + ":  " + res['_source']['text'])

{'took': 11,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 913, 'relation': 'eq'},
  'max_score': 56.10214,
  'hits': [{'_index': 'index',
    '_type': 'type',
    '_id': '510',
    '_score': 56.10214,
    '_source': {'text': '关于赫赛汀心脏毒性：1、临床 2、临床前',
     'processed_text': '心脏  心脏毒性  脏毒  毒性  临床  临床  前'}},
   {'_index': 'index',
    '_type': 'type',
    '_id': '620',
    '_score': 35.769,
    '_source': {'text': '[Wechat] 临床上赫赛汀q3W和qW用法，在心脏毒性方面，哪个频率心脏毒性小些？相关研究？',
     'processed_text': '临床  床上  qW  qW  用法  心脏  心脏毒性  脏毒  毒性  方面  频率  心脏  心脏毒性  脏毒  毒性  相关  研究'}},
   {'_index': 'index',
    '_type': 'type',
    '_id': '774',
    '_score': 23.778368,
    '_source': {'text': '[Wechat] 关于tdm1的临床试验',
     'processed_text': 'tdm  临床  临床试验  试验'}},
   {'_index': 'index',
    '_type': 'type',
    '_id': '1067',
    '_score': 23.509974,
    '_source': {'text': '[Wechat] 请帮忙查询以下临床研究的全文以供学习，谢谢！ 1、XT对比GT在晚期乳腺癌化疗中的临床研究原文；重点关注毒性的对比。